In [1]:
from utils.get_ticker_10k_filings import get_ticker_10k_filings
from utils.collect_ticker_files import collect_ticker_files
from utils.delete_txt_files import delete_txt_files
from utils.parse_html_file_mda import parse_html_file_mda
from shutil import rmtree
from dotenv import load_dotenv
import os
from supabase import create_client
from utils.new_10k_reports_to_supabase_mda import new_10k_reports_to_supabase_mda

In [2]:
ticker = 'AAPL'
try:
    get_ticker_10k_filings(ticker)
except Exception as e:
    print(f"Error occurred while downloading filings for {ticker}: {e}")


2023-12-29 17:56:37,595:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:56:39,601:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 4 items
2023-12-29 17:56:41,604:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 4 items
2023-12-29 17:56:43,610:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 4 items
2023-12-29 17:56:45,613:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 4 items
2023-12-29 17:56:47,618:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 5 items
2023-12-29 17:56:49,624:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 5 items


2023-12-29 17:56:51,629:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 6 items
2023-12-29 17:56:53,635:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:56:55,640:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:56:57,646:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:56:59,651:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:57:01,657:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:57:03,662:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17

In [3]:
ticker_files_dict = collect_ticker_files()
ticker_files_dict

Files are ready for AAPL


{'AAPL': ['data/sec-edgar-filings/AAPL/10-K/0001193125-08-224958/full-submission.txt',
  'data/sec-edgar-filings/AAPL/10-K/0001193125-08-224958/primary-document.html',
  'data/sec-edgar-filings/AAPL/10-K/0001104659-06-084288/full-submission.txt',
  'data/sec-edgar-filings/AAPL/10-K/0001104659-06-084288/primary-document.html',
  'data/sec-edgar-filings/AAPL/10-K/0000320193-19-000119/full-submission.txt',
  'data/sec-edgar-filings/AAPL/10-K/0000320193-19-000119/primary-document.html',
  'data/sec-edgar-filings/AAPL/10-K/0000320193-94-000016/full-submission.txt',
  'data/sec-edgar-filings/AAPL/10-K/0000912057-99-010244/full-submission.txt',
  'data/sec-edgar-filings/AAPL/10-K/0001193125-12-444068/full-submission.txt',
  'data/sec-edgar-filings/AAPL/10-K/0001193125-12-444068/primary-document.html',
  'data/sec-edgar-filings/AAPL/10-K/0000320193-17-000070/full-submission.txt',
  'data/sec-edgar-filings/AAPL/10-K/0000320193-17-000070/primary-document.html',
  'data/sec-edgar-filings/AAPL/10-

2023-12-29 17:57:17,726:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:57:19,732:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items


In [4]:
delete_txt_files(ticker_files_dict.get(ticker, []))


In [5]:
ticker_files_dict.get(ticker, [])[1]

'data/sec-edgar-filings/AAPL/10-K/0001193125-08-224958/primary-document.html'

2023-12-29 17:57:21,735:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items


In [6]:
all_parsed_data = {}
count = 0

html_file = ticker_files_dict.get(ticker, [])[1]

html_file

'data/sec-edgar-filings/AAPL/10-K/0001193125-08-224958/primary-document.html'

2023-12-29 17:57:23,740:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:57:25,743:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:57:27,749:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:57:29,754:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items


In [7]:
if html_file.endswith(".html"):
    path_parts = html_file.split("/")
    cik_year_acc = path_parts[4].split("-")

    if len(cik_year_acc) < 3:
        print(f"Skipping file with unexpected format: {html_file}")

    CIK = cik_year_acc[0]
    # Convert the two-digit year to four digits
    two_digit_year = cik_year_acc[1]
    if int(two_digit_year) > 50:  # Assuming we're starting from 1950 for simplicity
        Year = "19" + two_digit_year
    else:
        Year = "20" + two_digit_year
    AccessionNumber = cik_year_acc[2]
    try:
        parsed_data = parse_html_file_mda(html_file)
    except Exception as e:
        print(f"Could not parse {html_file} due to error: {e}")
parsed_data

'Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations This section and other parts of this Form 10-K contain forward-looking statements that involve risks and uncertainties. Forward-looking statements can also be identified by words such as “anticipates,” “expects,” “believes,” “plans,” “predicts,” and similar terms. Forward-looking statements are not guarantees of future performance and the Company’s actual results may differ significantly from the results discussed in the forward-looking statements. Factors that might cause such differences include, but are not limited to, those discussed in the subsection entitled “Risk Factors” above, which are incorporated herein by reference. The following discussion should be read in conjunction with the consolidated financial statements and notes thereto included in Item 8 of this Form 10-K. All information presented herein is based on the Company’s fiscal calendar. Unless otherwise stated, references in

2023-12-29 17:57:31,760:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:57:33,766:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:57:35,771:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:57:37,775:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:57:39,781:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:57:41,787:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:57:43,792:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17

In [8]:
try:
    filing_dict = {
        "ticker": ticker,
        "cik": CIK,  # PULL DATA FROM THE CUSTOM CIK DATA LOOKUP TO REDUCE JOINING ERRORS IN THE FUTURE
        "year": int(Year),
        "accession_number": AccessionNumber,
        "mda_section": parsed_data,
    }
except ValueError:
    print(f"Skipping file with invalid year format in {html_file}")

all_parsed_data[AccessionNumber] = filing_dict

all_parsed_data_list = list(all_parsed_data.values()) # list of dictionaries that hold all the scraped text form the 10k reports for a specific ticker. 

2023-12-29 17:59:38,092:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items


In [9]:
all_parsed_data_list

[{'ticker': 'AAPL',
  'cik': '0001193125',
  'year': 2008,
  'accession_number': '224958',
  'mda_section': 'Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations This section and other parts of this Form 10-K contain forward-looking statements that involve risks and uncertainties. Forward-looking statements can also be identified by words such as “anticipates,” “expects,” “believes,” “plans,” “predicts,” and similar terms. Forward-looking statements are not guarantees of future performance and the Company’s actual results may differ significantly from the results discussed in the forward-looking statements. Factors that might cause such differences include, but are not limited to, those discussed in the subsection entitled “Risk Factors” above, which are incorporated herein by reference. The following discussion should be read in conjunction with the consolidated financial statements and notes thereto included in Item 8 of this Form 10-K. All in

2023-12-29 17:59:42,098:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:59:44,104:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:59:46,110:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:59:48,115:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:59:50,121:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:59:52,127:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17:59:54,132:INFO - (sync)leaking bucket: <pyrate_limiter.buckets.in_memory_bucket.InMemoryBucket object at 0x110753c90>, 0 items
2023-12-29 17